# Video pre-processing

This notebook uses ffmpeg to convert the raw videos into a reliably seekable format that works best with SLEAP and crops the videos if necessary. Enter values for each variable in the first cell below according to your setup and requirements. Each variable and corresponding value should look like the following,

`variable_name = value`

After entering values for all variables, click Cell > Run All in the top menu bar to execute the notebook and process the videos.

Note that the 'r' before the opening quotation mark for the `ffmpeg_path`, `input_folder`, and `output_folder` variables is required to ensure that backslashes in the folder path don't cause issues during processing, as the backslash is usually used to denote an "escaped" character in character strings.

## Parameters

In [ ]:
# Enter the folder path for the extracted ffmpeg folder
# e.g. r"D:\Downloads\ffmpeg-6.1.1-essentials_build"
ffmpeg_path = r""

# Enter the folder path containing the raw video files
# e.g. r"D:\Downloads\Ant videos"
input_folder = r""

# Enter the folder path for the output folder where the processed videos should be saved
# e.g. r"D:\Downloads\Ant videos\processed"
output_folder = r""

# Enter a suffix that should be appended to the end of the filenames of the processed videos if required (e.g. "_proc")
# or leave as a blank string (i.e. "") if no suffix is required
output_suffix = "_proc"

### Video cropping settings

These settings control which section of the original video is cropped and saved to the output file if configured. The cropped region is defined by the offset of the top-left corner of the cropped region from the top-left corner of the video frame (`crop_box_topleft_x` and `crop_box_topleft_y`), and the width and height of the cropped region (`crop_box_width` and `crop_box_height`). Note that `crop_videos` must be set to `True` for these settings to have any effect.

For example, to crop a 1920x1080 video so the output consists only of the lower-left quadrant of the video, you would set the variables as follows,

- `crop_videos = True`
- `crop_box_topleft_x = 0`
- `crop_box_topleft_y = 540`
- `crop_box_width = 960`
- `crop_box_height = 540`

In [ ]:
# Set crop_videos to True (without quotation marks) if the processed videos should be cropped,
# or False if no cropping should be performed
crop_videos = False

# Set to the distance in pixels that the left edge of the crop region should be from the left edge of the video frame
crop_box_topleft_x = 0

# Set to the distance in pixels that the top edge of the crop region should be from the top edge of the video frame
crop_box_topleft_y = 0

# Set to the width of the crop region in pixels
crop_box_width = 0

# Set to the height of the crop region in pixels
crop_box_height = 0

## Code execution

In [ ]:
# Imports the Python modules required for the notebook to run
import os
import subprocess

# Recursively searches the input path for the ffmpeg.exe executable
def find_exe(path):
    for entry in os.scandir(path):
        if entry.is_file() and entry.name == 'ffmpeg.exe':
            return entry.path
        elif entry.is_dir():
            found_path = find_exe(entry.path)
            
            if found_path is not None:
                return found_path
    
    return None

# If the ffmpeg_path leads directly to ffmpeg.exe, check that the path is valid
if ffmpeg_path.endswith("ffmpeg.exe"):    
    if not os.path.exists(ffmpeg_path):
        raise FileNotFoundError(f"ffmpeg.exe not found in {ffmpeg_path}, check that the path is correct")
    
    ffmpeg_loc = ffmpeg_path
else:
    # Search the path for the ffmpeg.exe file
    ffmpeg_loc = find_exe(ffmpeg_path)
    
    if ffmpeg_loc is None:
        raise FileNotFoundError(f"ffmpeg.exe not found in {ffmpeg_path}, check that the correct folder has been set")

In [ ]:
# Check that the input and output folders exist
if not os.path.exists(input_folder):
    raise FileNotFoundError("Input folder not found, check that the input_folder path is correct")
    
if not os.path.exists(output_folder):
    raise FileNotFoundError("Output folder does not exist, check that the output_folder path is correct")

In [ ]:
# Process each video in the input folder and save it to the output folder
for entry in os.scandir(input_folder):
    if entry.is_file() and entry.name.lower().endswith('.mp4'):
        output_filename = os.path.join(output_folder, "".join(entry.name.split('.')[0:-1])) + output_suffix + ".mp4"
        params = [ffmpeg_loc, '-y', '-i', entry.path]
        
        if crop_videos:
            params.extend(['-vf', f'crop={crop_box_width}:{crop_box_height}:{crop_box_topleft_x}:{crop_box_topleft_y}'])
        
        params.extend(['-c:v', 'libx264', '-pix_fmt', 'yuv420p', '-preset', 'superfast', '-crf', '23', output_filename])
        process = subprocess.run(params)